<a href="https://colab.research.google.com/github/assafrbi/mini-project-Digital-Humanities/blob/main/dh_mini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install cinemagoer
!pip install wikipedia
import requests
from bs4 import BeautifulSoup
from dateutil.parser import parse
import concurrent.futures
import pandas as pd
from imdb import Cinemagoer
import wikipedia
import numpy as np

# Maximum number of threads that will be spawned
MAX_THREADS = 50

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Load Data from IMDb

In [ ]:
movie_title_arr = []
movie_year_arr = []
movie_genre_arr = []
image_id_arr = []

In [ ]:
def getMovieTitle(header):
    try:
        return header[0].find("a").getText()
    except:
        return 'NA'

def getReleaseYear(header):
    try:
        return header[0].find("span",  {"class": "lister-item-year text-muted unbold"}).getText()
    except:
        return 'NA'

def getGenre(muted_text):
    try:
        return muted_text.find("span",  {"class":  "genre"}).getText().strip()
    except:
        return 'NA'

def getImageId(image):
    try:
        return image.get('data-tconst')[2:]
    except:
        return 'NA'

In [ ]:
def main(imdb_url):
    response = requests.get(imdb_url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Movie Name
    movies_list  = soup.find_all("div", {"class": "lister-item mode-advanced"})
    
    for movie in movies_list:
        header = movie.find_all("h3", {"class":  "lister-item-header"})
        muted_text = movie.find_all("p", {"class":  "text-muted"})[0]
        imageDiv =  movie.find("div", {"class": "lister-item-image float-left"})
        image = imageDiv.find("img", "loadlate")
        
        # Movie Title
        movie_title = getMovieTitle(header)
        movie_title_arr.append(movie_title)

        # Movie id
        id = getImageId(image)
        image_id_arr.append(id)
        
        # Movie release year
        year = getReleaseYear(header)
        movie_year_arr.append(year)
        
        # Genre
        genre = getGenre(muted_text)
        movie_genre_arr.append(genre)
        
        # # Movie Synopsis
        # synopsis = getsynopsis(id)
        # movie_synopsis_arr.append(synopsis)
        

In [ ]:
# An array to store all the URL that are being queried
imageArr = []

# Maximum number of pages one wants to iterate over
MAX_PAGE =51

# Loop to generate all the URLS.
for i in range(1, 1500, 50):
    imdb_url = f'https://www.imdb.com/search/title/?title_type=feature&countries=il&start={i}&ref_=adv_nxt'
    imageArr.append(imdb_url)

In [ ]:
def download_stories(story_urls):
    threads = min(MAX_THREADS, len(story_urls))
    with concurrent.futures.ThreadPoolExecutor(max_workers=threads) as executor:
        executor.map(main, story_urls)

In [ ]:
# Call the download function with the array of URLS called imageArr
download_stories(imageArr)

# Attach all the data to the pandas dataframe. You can optionally write it to a CSV file as well
movieDf = pd.DataFrame({
    "Title": movie_title_arr,
    "Year": movie_year_arr,
    "Genre": movie_genre_arr,
    "IMDb id": image_id_arr,
})

movieDf.to_csv('israel_originated_movies_imdb.csv', index=False)
print('--------- Download Complete CSV Formed --------')

movieDf

--------- Download Complete CSV Formed --------


,Title,Year,Genre,IMDb id
0,The Lost Daughter,(2021),Drama,9100054
1,Knock Knock,(I) (2015),"Drama, Thriller",3605418
2,Cobra,(1986),"Action, Crime, Thriller",0090859
3,The Delta Force,(1986),"Action, Adventure, Drama",0090927
4,The Kindergarten Teacher,(2018),"Drama, Thriller",6952960
...,...,...,...,...
1466,Double Edge,(1992),"Drama, Thriller",0104131
1467,The Loners,(2009),Drama,1455618
1468,A Place in Heaven,(2013),Drama,2960994
1469,The Spy Who Never Was,(1976),Drama,0070812


### Load Synopses from IMDb

In [15]:
imdb_synopses_arr = []

def getImdbSynopsis(id):
  ia = Cinemagoer()
  movie = ia.get_movie(id)
  imdb_synopses_arr.append(movie['plot'])

def get_from_imdb(ids):
  threads = min(MAX_THREADS, len(ids))
  with concurrent.futures.ThreadPoolExecutor(max_workers=threads) as executor:
      executor.map(getImdbSynopsis, ids)

In [16]:
data = pd.read_csv('israel_originated_movies_imdb.csv')
image_id_arr = data['IMDb id'].values

In [17]:
get_from_imdb(image_id_arr)
data['Synopsis'] = imdb_synopses_arr

data.to_csv('israel_originated_movies_imdb.csv', index=False)
print('--------- Download Complete CSV Formed --------')

data

Streaming output truncated to the last 5000 lines.
    response = uopener.open(url)
  File "/usr/lib/python3.7/urllib/request.py", line 531, in open
    response = meth(req, response)
  File "/usr/lib/python3.7/urllib/request.py", line 641, in http_response
    'http', request, response, code, msg, hdrs)
  File "/usr/lib/python3.7/urllib/request.py", line 569, in error
    return self._call_chain(*args)
  File "/usr/lib/python3.7/urllib/request.py", line 503, in _call_chain
    result = func(*args)
  File "/usr/lib/python3.7/urllib/request.py", line 649, in http_error_default
    raise HTTPError(req.full_url, code, msg, hdrs, fp)
urllib.error.HTTPError: HTTP Error 503: Service Unavailable

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/imdb/__init__.py", line 831, in update
    ret = method(mopID)
  File "/usr/local/lib/python3.7/dist-packages/imdb/parser/http/__init__.py", line 468,

KeyboardInterrupt: ignored

Streaming output truncated to the last 5000 lines.
    cont = self._retrieve(self.urls['movie_main'] % movieID + 'reference')
  File "/usr/local/lib/python3.7/dist-packages/imdb/parser/http/__init__.py", line 379, in _retrieve
    ret = self.urlOpener.retrieve_unicode(url, size=size)
  File "/usr/local/lib/python3.7/dist-packages/imdb/parser/http/__init__.py", line 255, in retrieve_unicode
    'original exception': e}
imdb._exceptions.IMDbDataAccessError: {'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/title/tt13875978/reference', 'proxy': '', 'exception type': 'IOError', 'original exception': <HTTPError 503: 'Service Unavailable'>}
2022-07-11 18:49:38,215 CRITICAL [imdbpy] /usr/local/lib/python3.7/dist-packages/imdb/_exceptions.py:34: IMDbDataAccessError exception raised; args: ({'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/title/tt1740807/reference', 'proxy': '', 'exception type': 'IOError', 'original exception': <HTTPError 503: 'Service Unavailabl

### Load Synopses from Wikipedia - Need some tuning !!!!!!

In [ ]:
movie_plot_arr = []
movie_summary_arr = []
wikipedia_title_arr = []

def find_title_on_wikipedia(title, year):
    search_results = wikipedia.search(title, results=20)
    search_results_exact = [s for s in search_results if title in s] # results that have the exact title as part of them
    
    if len(search_results_exact) != 0: # if the title is part of at least one result

        # if film exists in title but not together with series or score (an example was Resident Evil 2002)
        search_results_film = [s for s in search_results_exact if 'film' in s and 'score' not in s and 'series' not in s]

        # Go on by additionaly searching for the year in the title
        if len(search_results_film) == 0: # if there is no results with 'film' in the title
            search_results_year = [s for s in search_results_exact if str(year) in s]
            if len(search_results_year) == 0: # if there is no results with year in the title
                search_results_final = search_results_exact
            else:
                search_results_final = search_results_year
        else:
            search_results_year = [s for s in search_results_film if str(year) in s]
            if len(search_results_year) == 0:
                search_results_final = search_results_film
            else:
                search_results_final = search_results_year  

        # for the possible remains of the search results take the exact match or the first result
        for s in search_results_final:
            if s == title:
                title = s
            else:
                title = search_results_final[0]
    
    else: # take the first result
        title = search_results[0]

    return title

def add_wikipedia():
  section_names = ['Plot','Synopsis','Plot synopsis','Plot summary', 'Story','Plotline','The Beginning','Summary', 'Content','Premise']
  for i in range (len(image_id_arr)):
    imdb_title = movie_title_arr[i]
    year = movie_year_arr[i]
    title, page, plot, summary = np.nan, np.nan, np.nan, np.nan

    try:
      title = find_title_on_wikipedia(imdb_title, year)
      search_results = wikipedia.search(title, results=20)
      page = wikipedia.WikipediaPage(title = title)
    except:
        print(f'Title "{imdb_title}" does not exist on Wikipedia.')

    for section in section_names:
        try:
            plot = page.section(section)
        except:
            print(f'Plot for -{imdb_title}- was not found.')

    # for some movies with general title - modify the search by adding (year film) next to title
    if plot is None:
        imdb_title = f'{imdb_title} ({year} film)'
        try:          
            title = find_title_on_wikipedia(imdb_title, year)
            search_results = wikipedia.search(title, results=20)
            page = wikipedia.WikipediaPage(title = title)
        except:
            print(f'Title "{imdb_title}" does not exist on Wikipedia.')

        for section in section_names:
            try:
                plot = page.section(section)
            except:
              print(f'Plot for -{imdb_title}- was not found.')
        
    try:
        summary = page.summary
    except:
        print(f'Summary for -{imdb_title}- was not found.')        
    
    wikipedia_title_arr.append(title)
    movie_plot_arr.append(plot)
    movie_summary_arr.append(summary)

In [ ]:
data = pd.read_csv('israel_originated_movies_imdb.csv')
add_wikipedia()

/usr/local/lib/python3.7/dist-packages/wikipedia/wikipedia.py:389: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.7/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Title "Norman" does not exist on Wikipedia.
Plot for -Norman- was not found.
Plot for -Norman- was not found.
Plot for -Norman- was not found.
Plot for -Norman- was not found.
Plot for -Norman- was not found.
Plot for -Norman- was not found.
Plot for -Norman- was not found.
Plot for -Norman- was not found.
Plot for -Norman- was not found.
Plot for -Norman- was not found.
Summary for -Norman- was not found.


/usr/local/lib/python3.7/dist-packages/wikipedia/wikipedia.py:386: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 386 of the file /usr/local/lib/python3.7/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  request = _wiki_request(query_params)


Title "The Body" does not exist on Wikipedia.
Plot for -The Body- was not found.
Plot for -The Body- was not found.
Plot for -The Body- was not found.
Plot for -The Body- was not found.
Plot for -The Body- was not found.
Plot for -The Body- was not found.
Plot for -The Body- was not found.
Plot for -The Body- was not found.
Plot for -The Body- was not found.
Plot for -The Body- was not found.
Summary for -The Body- was not found.
Title "Jean Wejnoon" does not exist on Wikipedia.
Plot for -Jean Wejnoon- was not found.
Plot for -Jean Wejnoon- was not found.
Plot for -Jean Wejnoon- was not found.
Plot for -Jean Wejnoon- was not found.
Plot for -Jean Wejnoon- was not found.
Plot for -Jean Wejnoon- was not found.
Plot for -Jean Wejnoon- was not found.
Plot for -Jean Wejnoon- was not found.
Plot for -Jean Wejnoon- was not found.
Plot for -Jean Wejnoon- was not found.
Summary for -Jean Wejnoon- was not found.


/usr/local/lib/python3.7/dist-packages/wikipedia/wikipedia.py:386: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 386 of the file /usr/local/lib/python3.7/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  request = _wiki_request(query_params)


...

Title "Nelson" does not exist on Wikipedia.
Plot for -Nelson- was not found.
Plot for -Nelson- was not found.
Plot for -Nelson- was not found.
Plot for -Nelson- was not found.
Plot for -Nelson- was not found.
Plot for -Nelson- was not found.
Plot for -Nelson- was not found.
Plot for -Nelson- was not found.
Plot for -Nelson- was not found.
Plot for -Nelson- was not found.
Summary for -Nelson- was not found.


/usr/local/lib/python3.7/dist-packages/wikipedia/wikipedia.py:386: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 386 of the file /usr/local/lib/python3.7/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  request = _wiki_request(query_params)


Title "Restoration" does not exist on Wikipedia.
Plot for -Restoration- was not found.
Plot for -Restoration- was not found.
Plot for -Restoration- was not found.
Plot for -Restoration- was not found.
Plot for -Restoration- was not found.
Plot for -Restoration- was not found.
Plot for -Restoration- was not found.
Plot for -Restoration- was not found.
Plot for -Restoration- was not found.
Plot for -Restoration- was not found.
Summary for -Restoration- was not found.


/usr/local/lib/python3.7/dist-packages/wikipedia/wikipedia.py:386: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 386 of the file /usr/local/lib/python3.7/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  request = _wiki_request(query_params)


Title "The Forest" does not exist on Wikipedia.
Plot for -The Forest- was not found.
Plot for -The Forest- was not found.
Plot for -The Forest- was not found.
Plot for -The Forest- was not found.
Plot for -The Forest- was not found.
Plot for -The Forest- was not found.
Plot for -The Forest- was not found.
Plot for -The Forest- was not found.
Plot for -The Forest- was not found.
Plot for -The Forest- was not found.
Summary for -The Forest- was not found.
Title "The Journey to Planet Minimik" does not exist on Wikipedia.
Plot for -The Journey to Planet Minimik- was not found.
Plot for -The Journey to Planet Minimik- was not found.
Plot for -The Journey to Planet Minimik- was not found.
Plot for -The Journey to Planet Minimik- was not found.
Plot for -The Journey to Planet Minimik- was not found.
Plot for -The Journey to Planet Minimik- was not found.
Plot for -The Journey to Planet Minimik- was not found.
Plot for -The Journey to Planet Minimik- was not found.
Plot for -The Journey to Pl

### Text Preprocessing


### Topic Modeling - Latent Dirichlet Allocation